In [1]:
%run prepareStats.py

Last GP in the database: the 2023 Bahrain Grand Prix


In [2]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [3]:
results.groupby(["surname","forename"])["forename"].nunique()

surname                  forename         
Abate                    Carlo                1
Abecassis                George               1
Acheson                  Kenny                1
Adams                    Philippe             1
Ader                     Walt                 1
Adolff                   Kurt                 1
Agabashian               Fred                 1
Ahrens                   Kurt                 1
Aitken                   Jack                 1
Albers                   Christijan           1
Albon                    Alexander            1
Alboreto                 Michele              1
Alesi                    Jean                 1
Alguersuari              Jaime                1
Alliot                   Philippe             1
Allison                  Cliff                1
Alonso                   Fernando             1
Amati                    Giovanna             1
Amick                    George               1
                         Red                 

In [4]:
fathersAndSons = {
"Andrettis": ["Mario Andretti", "Michael Andretti"],
"Brabhams-D": ["Jack Brabham", "David Brabham"],
"Brabhams-G": ["Jack Brabham", "Gary Brabham"],
"Fittipaldis": ["Wilson Fittipaldi", "Christian Fittipaldi"],
"Hills": ["Graham Hill", "Damon Hill"], 
"Magnussens": ["Jan Magnussen", "Kevin Magnussen"],
"Nakajimas": ["Satoru Nakajima", "Kazuki Nakajima"],
"Palmers": ["Jonathan Palmer", "Jolyon Palmer"],
"Parnells": ["Reg Parnell", "Tim Parnell"],
"Pilletes": ["André Pilette", "Teddy Pilette"],
"Piquets": ["Nelson Piquet", "Nelson Piquet Jr."], 
"Rosbergs": ["Keke Rosberg", "Nico Rosberg"],
"Stucks": ["Hans von Stuck","Hans-Joachim Stuck"],
"Schumachers": ["Michael Schumacher", "Mick Schumacher"],
"Verstappens": ["Jos Verstappen", "Max Verstappen"], 
"Villeneuves": ["Gilles Villeneuve", "Jacques Villeneuve"],
"Winkelhocks": ["Manfred Winkelhock", "Markus Winkelhock"]
}

In [5]:
len(fathersAndSons)

17

There have been 17 F1 drivers that are sons of another F1 driver. (There is still a small chance I missed some obscure family tie from the early eras.)

[María de Villota](https://en.wikipedia.org/wiki/Mar%C3%ADa_de_Villota) who died (most likely) due to injuries sustained while testing Marussia Formula One car was a daughter of F1 driver [Emilio de Villota](https://en.wikipedia.org/wiki/Emilio_de_Villota).

Also, several F1 drivers were descendants of pre-war Grand Prix drivers, [Alberto Ascari](https://en.wikipedia.org/wiki/Alberto_Ascari) and his father [Antonio Ascari](https://en.wikipedia.org/wiki/Antonio_Ascari) being probably the most notable example.

By participating in 2 races in 2020, [Pietro Fittipaldi](https://en.wikipedia.org/wiki/Pietro_Fittipaldi) became the first (and still the only) competing grandson of another driver.

In [6]:
sons = []
for key, value in fathersAndSons.items():
    sons.append(value[1])

In [7]:
results.loc[results.name.isin(sons),"son"]=True

In [8]:
results.loc[results.position.eq(1),"twentyFivePointSystem"]=25
results.loc[results.position.eq(2),"twentyFivePointSystem"]=18
results.loc[results.position.eq(3),"twentyFivePointSystem"]=15
results.loc[results.position.eq(4),"twentyFivePointSystem"]=12
results.loc[results.position.eq(5),"twentyFivePointSystem"]=10
results.loc[results.position.eq(6),"twentyFivePointSystem"]=8
results.loc[results.position.eq(7),"twentyFivePointSystem"]=6
results.loc[results.position.eq(8),"twentyFivePointSystem"]=4
results.loc[results.position.eq(9),"twentyFivePointSystem"]=2
results.loc[results.position.eq(10),"twentyFivePointSystem"]=1
results.loc[~results.position.isin([1,2,3,4,5,6,7,8,9,10]),"twentyFivePointSystem"]=0

In [9]:
resultsSons = results[results.name.isin(sons)]

## Historical stats

In [10]:
results[results["son"] == True].groupby("name")["date"].min().nsmallest(20)

name
Tim Parnell            1959-07-18
Hans-Joachim Stuck     1974-01-13
Teddy Pilette          1974-05-12
Gary Brabham           1990-03-11
David Brabham          1990-05-13
Christian Fittipaldi   1992-03-01
Damon Hill             1992-05-03
Michael Andretti       1993-03-14
Jacques Villeneuve     1996-03-10
Nico Rosberg           2006-03-12
Markus Winkelhock      2007-07-22
Kazuki Nakajima        2007-10-21
Nelson Piquet Jr.      2008-03-16
Kevin Magnussen        2014-03-16
Max Verstappen         2015-03-15
Jolyon Palmer          2016-03-20
Mick Schumacher        2021-03-28
Name: date, dtype: datetime64[ns]

In [11]:
woSons = results.sort_values(by="date").groupby(["year", "round", "location"])["son"].count()
woSons[woSons <1].tail(1)

year  round  location
2004  15     Monza       0
Name: son, dtype: int64

The last GP without a single son of another F1 driver entering was the 2004 Italian Grand Prix.

In [12]:
woSons = results.sort_values(by="date").groupby(["year"])["son"].count()
woSons[woSons <1].tail(1)

year
1991    0
Name: son, dtype: int64

In [13]:
results[results.name.isin(sons)].reset_index().groupby("year")["name"].nunique(list)

year
1959    1
1961    1
1963    1
1974    2
1975    1
1976    1
1977    2
1978    1
1979    1
1990    2
1992    2
1993    3
1994    3
1995    1
1996    2
1997    2
1998    2
1999    2
2000    1
2001    1
2002    1
2003    1
2004    1
2005    1
2006    2
2007    3
2008    3
2009    3
2010    1
2011    1
2012    1
2013    1
2014    2
2015    3
2016    4
2017    3
2018    2
2019    2
2020    2
2021    2
2022    3
2023    2
Name: name, dtype: int64

The 2016 season saw the most sons of another F1 driver participating in the WDC.

In [14]:
resultsSons[resultsSons["year"] == 2016].groupby("name")["raceId"].nunique()

name
Jolyon Palmer      21
Kevin Magnussen    21
Max Verstappen     21
Nico Rosberg       21
Name: raceId, dtype: int64

## Sons, ranked

In [15]:
resultsSons.groupby("name")["points"].sum().sort_values(ascending=False)

name
Max Verstappen          2008.5
Nico Rosberg            1594.5
Damon Hill               360.0
Jacques Villeneuve       235.0
Kevin Magnussen          179.0
Hans-Joachim Stuck        29.0
Nelson Piquet Jr.         19.0
Mick Schumacher           12.0
Christian Fittipaldi      12.0
Kazuki Nakajima            9.0
Jolyon Palmer              9.0
Michael Andretti           7.0
Markus Winkelhock          0.0
Gary Brabham               0.0
David Brabham              0.0
Teddy Pilette              0.0
Tim Parnell                0.0
Name: points, dtype: float64

In [16]:
resultsSons.groupby("name")["twentyFivePointSystem"].sum().sort_values(ascending=False)

name
Max Verstappen          2005.0
Nico Rosberg            1739.0
Damon Hill              1091.0
Jacques Villeneuve       853.0
Kevin Magnussen          179.0
Hans-Joachim Stuck       165.0
Christian Fittipaldi     100.0
Nelson Piquet Jr.         50.0
Michael Andretti          41.0
Kazuki Nakajima           34.0
Mick Schumacher           12.0
Jolyon Palmer              9.0
David Brabham              1.0
Tim Parnell                1.0
Markus Winkelhock          0.0
Gary Brabham               0.0
Teddy Pilette              0.0
Name: twentyFivePointSystem, dtype: float64

In [17]:
resultsSons[resultsSons["position"] == 1].groupby("name")["raceId"].nunique().sort_values(ascending=False)

name
Max Verstappen        36
Nico Rosberg          23
Damon Hill            22
Jacques Villeneuve    11
Name: raceId, dtype: int64

In [18]:
resultsSons.groupby("name")["raceId"].count().sort_values(ascending=False)

name
Nico Rosberg            206
Jacques Villeneuve      165
Max Verstappen          164
Kevin Magnussen         143
Damon Hill              122
Hans-Joachim Stuck       81
Mick Schumacher          44
Christian Fittipaldi     43
Jolyon Palmer            37
Kazuki Nakajima          36
David Brabham            30
Nelson Piquet Jr.        28
Michael Andretti         13
Teddy Pilette             4
Tim Parnell               4
Gary Brabham              2
Markus Winkelhock         1
Name: raceId, dtype: int64

In [19]:
resultsSons.groupby("name")["position"].min().sort_values()

name
Damon Hill               1.0
Jacques Villeneuve       1.0
Max Verstappen           1.0
Nico Rosberg             1.0
Kevin Magnussen          2.0
Nelson Piquet Jr.        2.0
Hans-Joachim Stuck       3.0
Michael Andretti         3.0
Christian Fittipaldi     4.0
Jolyon Palmer            6.0
Kazuki Nakajima          6.0
Mick Schumacher          6.0
David Brabham           10.0
Tim Parnell             10.0
Teddy Pilette           17.0
Gary Brabham             NaN
Markus Winkelhock        NaN
Name: position, dtype: float64

In [20]:
resultsSons[resultsSons["grid"] == 1].groupby("name")["raceId"].count().sort_values(ascending=False)

name
Nico Rosberg          30
Max Verstappen        21
Damon Hill            20
Jacques Villeneuve    13
Name: raceId, dtype: int64

## Gaps between father's last GP and son's first

In [21]:
def timedelta():
    timedeltas = {}
    for key, value in fathersAndSons.items():
        delta = results[results["name"] == value[1]]["date"].min() - results[results["name"] == value[0]]["date"].max()
        delta = delta
        timedeltas[key] = delta
    timedeltas = pd.DataFrame.from_dict(timedeltas, orient="index", columns = ["gap"])    
    return(timedeltas)
        
gap = timedelta().sort_values(by="gap")
gap

,gap
Parnells,1827 days
Schumachers,3045 days
Pilletes,3570 days
Andrettis,3823 days
Verstappens,4172 days
Villeneuves,5054 days
Magnussens,5761 days
Nakajimas,5831 days
Piquets,5978 days
Fittipaldis,5992 days


In [22]:
def seasondelta():
    seasondeltas = {}
    for key, value in fathersAndSons.items():
        delta = []
        x = results[results["name"] == value[0]]["year"].max()
        delta.append(x)
        y = results[results["name"] == value[1]]["year"].min()
        delta.append(y)
        z = y - (x + 1)
        delta.append(z)
        seasondeltas[key] = delta
    seasondeltas = pd.DataFrame.from_dict(seasondeltas, orient="index", columns = ["last","first","gap"])    
    return(seasondeltas)
        
seasongap = seasondelta().sort_values(by="gap")
seasongap

,last,first,gap
Parnells,1954,1959,4
Schumachers,2012,2021,8
Pilletes,1964,1974,9
Andrettis,1982,1993,10
Verstappens,2003,2015,11
Villeneuves,1982,1996,13
Magnussens,1998,2014,15
Nakajimas,1991,2007,15
Hills,1975,1992,16
Piquets,1991,2008,16


## Drivers both father and son have raced against

In [23]:
results["adversaries"] = results["raceId"].map(results.groupby("raceId")["name"].agg(list))

In [24]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

def adversaries():
    adversaries = {}
    for key, value in fathersAndSons.items():
        father = results[results["name"] == value[0]]
        father = father["adversaries"].tolist()
        father = [item for sublist in father for item in sublist]
        father = list(set(father))
        son = results[results["name"] == value[1]]
        son = son["adversaries"].tolist()
        son = [item for sublist in son for item in sublist]
        son = list(set(son))        
        inter = intersection(father, son)
        pair = [len(inter), ", ".join(inter)]
        adversaries[key] = pair
    pd.set_option('display.max_colwidth', None)
    adversaries = pd.DataFrame.from_dict(adversaries, orient="index", columns=["count", "names"])    
    
    return(adversaries)

commonAdversaries = pd.DataFrame(adversaries().sort_values(by="count", ascending = False))
commonAdversaries

,count,names
Schumachers,8,"Nico Hülkenberg, Fernando Alonso, Lewis Hamilton, Kimi Räikkönen, Daniel Ricciardo, Sergio Pérez, Robert Kubica, Sebastian Vettel"
Parnells,6,"André Pilette, Harry Schell, Maurice Trintignant, Roy Salvadori, Ron Flockhart, Stirling Moss"
Andrettis,5,"Riccardo Patrese, Michele Alboreto, Andrea de Cesaris, Derek Warwick, Alain Prost"
Verstappens,3,"Fernando Alonso, Kimi Räikkönen, Jenson Button"
Pilletes,2,"Graham Hill, Mike Hailwood"
Fittipaldis,0,
Hills,0,
Magnussens,0,
Nakajimas,0,
Palmers,0,


## Circuits both father and son have raced (and won) at

In [25]:
results["tracks"] = results["name"].map(results.groupby("name")["location"].agg(list))

In [26]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

def tracks():
    tracks = {}
    for key, value in fathersAndSons.items():
        father = results[results["name"] == value[0]]
        father = father["tracks"].tolist()
        father = [item for sublist in father for item in sublist]
        father = list(set(father))
        son = results[results["name"] == value[1]]
        son = son["tracks"].tolist()
        son = [item for sublist in son for item in sublist]
        son = list(set(son))        
        inter = intersection(father, son)
        pair = [len(inter), ", ".join(inter)]
        tracks[key] = pair
    pd.set_option('display.max_colwidth', None)
    tracks = pd.DataFrame.from_dict(tracks, orient="index", columns=["count", "tracks"])    
    
    return(tracks)

commonCirctuis = pd.DataFrame(tracks().sort_values(by="count", ascending = False))
commonCirctuis

,count,tracks
Schumachers,18,"Suzuka, Austin, Montreal, Monte-Carlo, Montmeló, Spielberg, Mexico City, Monza, Abu Dhabi, Budapest, Spa, Silverstone, Melbourne, Istanbul, Imola, Marina Bay, Sakhir, São Paulo"
Verstappens,15,"Kuala Lumpur, Suzuka, Montreal, Monte-Carlo, Montmeló, Spielberg, Monza, Budapest, Nürburg, Spa, Silverstone, Hockenheim, Melbourne, Imola, São Paulo"
Magnussens,14,"Monza, Suzuka, Montreal, Budapest, Nürburg, Imola, Spa, São Paulo, Melbourne, Silverstone, Monte-Carlo, Montmeló, Hockenheim, Spielberg"
Rosbergs,12,"Montreal, Monte-Carlo, Spielberg, Mexico City, Monza, Budapest, Nürburg, Spa, Silverstone, Hockenheim, Imola, São Paulo"
Nakajimas,11,"Suzuka, Montreal, Monte-Carlo, Montmeló, Magny Cours, Monza, Budapest, Spa, Silverstone, Hockenheim, São Paulo"
Piquets,11,"Montreal, Monte-Carlo, Montmeló, Magny Cours, Monza, Budapest, Nürburg, Spa, Silverstone, Hockenheim, São Paulo"
Hills,10,"Monte-Carlo, Spielberg, Monza, Nürburg, Spa, Silverstone, Buenos Aires, Hockenheim, São Paulo, Midrand"
Palmers,10,"Suzuka, Montreal, Monte-Carlo, Spielberg, Mexico City, Monza, Budapest, Spa, Silverstone, Hockenheim"
Villeneuves,9,"Montreal, Monte-Carlo, Spielberg, Monza, Silverstone, Buenos Aires, Hockenheim, Imola, São Paulo"
Andrettis,8,"Montreal, Monte-Carlo, Monza, Silverstone, Hockenheim, Imola, São Paulo, Midrand"


In [27]:
wins["tracks"] = wins["name"].map(wins.groupby("name")["location"].agg(list))

C:\Users\micha\AppData\Local\Temp\ipykernel_4960\1730858596.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wins["tracks"] = wins["name"].map(wins.groupby("name")["location"].agg(list))


In [28]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

def tracks():
    tracks = {}
    for key, value in fathersAndSons.items():
        father = wins[results["name"] == value[0]]
        father = father["tracks"].dropna().tolist()
        father = [item for sublist in father for item in sublist]
        father = list(set(father))
        son = wins[results["name"] == value[1]]
        son = son["tracks"].dropna().tolist()
        son = [item for sublist in son for item in sublist]
        son = list(set(son))        
        inter = intersection(father, son)
        pair = [len(inter), ", ".join(inter)]
        tracks[key] = pair
    pd.set_option('display.max_colwidth', None)
    tracks = pd.DataFrame.from_dict(tracks, orient="index", columns=["count", "tracks"])    
    
    return(tracks)

tracks().sort_values(by="count", ascending = False)

C:\Users\micha\AppData\Local\Temp\ipykernel_4960\2317356391.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  father = wins[results["name"] == value[0]]
C:\Users\micha\AppData\Local\Temp\ipykernel_4960\2317356391.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  son = wins[results["name"] == value[1]]


,count,tracks
Hills,1,Monza
Rosbergs,1,Monte-Carlo
Andrettis,0,
Pilletes,0,
Villeneuves,0,
Verstappens,0,
Schumachers,0,
Stucks,0,
Piquets,0,
Parnells,0,


## Back into father's team

In [29]:
commonTeams = {}
for key, value in fathersAndSons.items():
    fathersTeams = results[results["name"] == value[0]]["constructor"].drop_duplicates().to_list()
    sonsTeams = results[results["name"] == value[1]]["constructor"].drop_duplicates().to_list()
    teams = []
    for x in sonsTeams:
        if x in fathersTeams:
            teams.append(x)
    commonTeams[key] = teams
commonTeams = pd.DataFrame.from_dict(commonTeams, orient="index", columns=["shared teams"])
commonTeams

,shared teams
Andrettis,None
Brabhams-D,Brabham
Brabhams-G,None
Fittipaldis,None
Hills,Brabham
Magnussens,McLaren
Nakajimas,None
Palmers,None
Parnells,None
Pilletes,None


## How old was the son on the day of his father's last race?

In [30]:
def timedeltaAge():
    timedeltasAge = {}
    for key, value in fathersAndSons.items():
        deltaAge = results[results["name"] == value[0]]["date"].max() - results[results["name"] == value[1]]["dob"].min()
        deltaAge = deltaAge
        timedeltasAge[key] = deltaAge
    timedeltasAge = pd.DataFrame.from_dict(timedeltasAge, orient="index", columns = ["age"])    
    return(timedeltasAge)
        
timedeltaAge().sort_values(by="age")

,age
Palmers,-441 days
Rosbergs,486 days
Stucks,986 days
Fittipaldis,1721 days
Brabhams-D,1876 days
Winkelhocks,1878 days
Magnussens,2071 days
Verstappens,2203 days
Piquets,2292 days
Nakajimas,2487 days


## iROZHLAS.cz article stuff

In [31]:
cesky = {
"Andrettis": "Mario a Michael Andretti",
"Brabhams-D": "Jack a David Brabhamovi",
"Brabhams-G": "Jack a Gary Brabhamovi",
"Fittipaldis": "Wilson a Christian Fittipaldi",
"Hills": "Graham a Damon Hillovi", 
"Magnussens": "Jan a Kevin Magnussenovi",
"Nakajimas": "Satoru a Kazuki Nakajimovi",
"Palmers": "Jonathan a Jolyon Palmerovi",
"Parnells": "Reg a Tim Parnellovi",
"Pilletes": "André a Teddy Pilettovi",
"Piquets": "Nelson a Nelsinho Piquetovi", 
"Rosbergs": "Keke a Nico Rosbergovi",
"Stucks": "Hans a Hans-Joachim Stuckovi",
"Schumachers": "Michael a Mick Schumacherovi",
"Verstappens": "Jos a Max Verstappenovi", 
"Villeneuves": "Gilles a Jacques Villeneuvovi",
"Winkelhocks": "Manfred a Markus Winkelhockovi"
}

In [32]:
from IPython.display import HTML

In [33]:
sTituly = {"Max Verstappen" : 2, "Damon Hill" : 1, "Nico Rosberg" : 1, "Jacques Villeneuve" : 1}
sDebut = pd.Series(resultsSons.groupby("name")["date"].min().sort_values(ascending=False), name="debut")
sEnd = pd.Series(resultsSons.groupby("name")["date"].max().sort_values(ascending=False), name="derniéra")
sTitles =pd.Series(sTituly, name="titulů")
sWins = pd.Series(resultsSons[resultsSons["position"] == 1].groupby("name")["raceId"].nunique().sort_values(), name="vítězství")
sPoles = pd.Series(resultsSons[resultsSons["grid"] == 1].groupby("name")["raceId"].nunique().sort_values(), name="poles")
sPoints = pd.Series(resultsSons.groupby("name")["points"].sum(), name="bodů")
sBest = pd.Series(resultsSons.groupby("name")["position"].min(), name="top výsledek")
sRaces = pd.Series(resultsSons.groupby("name")["raceId"].count(), name="závodů")

In [34]:
sonsFrame = pd.concat([sRaces,sTitles,sPoints,sWins,sPoles,sBest],axis=1).fillna(value=0).sort_values(by=["vítězství","závodů"], ascending=False)

In [35]:
sonsFrame["top výsledek"] = sonsFrame["top výsledek"].astype(str)
sonsFrame["top výsledek"] = sonsFrame["top výsledek"].str[:-1]

In [36]:
sonsFrame

,závodů,titulů,bodů,vítězství,poles,top výsledek
Max Verstappen,164,2.0,2008.5,36.0,21.0,1.
Nico Rosberg,206,1.0,1594.5,23.0,30.0,1.
Damon Hill,122,1.0,360.0,22.0,20.0,1.
Jacques Villeneuve,165,1.0,235.0,11.0,13.0,1.
Kevin Magnussen,143,0.0,179.0,0.0,0.0,2.
Hans-Joachim Stuck,81,0.0,29.0,0.0,0.0,3.
Mick Schumacher,44,0.0,12.0,0.0,0.0,6.
Christian Fittipaldi,43,0.0,12.0,0.0,0.0,4.
Jolyon Palmer,37,0.0,9.0,0.0,0.0,6.
Kazuki Nakajima,36,0.0,9.0,0.0,0.0,6.


In [37]:
fsCircuits = pd.Series(commonCirctuis["count"],name="společných okruhů")
fsAdversaries = pd.Series(commonAdversaries["count"],name="společných soupeřů")
fsGap = pd.Series(seasongap["gap"], name="sezón rozestupu")
fsLast = pd.Series(seasongap["last"], name="derniéra otce")
fsFirst = pd.Series(seasongap["first"], name="debut syna")

In [38]:
fsFrame = pd.concat([fsLast,fsFirst,fsGap,fsCircuits,fsAdversaries],axis=1).sort_values(by="debut syna",ascending=False)
fsFrame = fsFrame.rename(index=cesky)

In [39]:
fsFrame

,derniéra otce,debut syna,sezón rozestupu,společných okruhů,společných soupeřů
Michael a Mick Schumacherovi,2012,2021,8,18,8
Jonathan a Jolyon Palmerovi,1989,2016,26,10,0
Jos a Max Verstappenovi,2003,2015,11,15,3
Jan a Kevin Magnussenovi,1998,2014,15,14,0
Nelson a Nelsinho Piquetovi,1991,2008,16,11,0
Manfred a Markus Winkelhockovi,1985,2007,21,1,0
Satoru a Kazuki Nakajimovi,1991,2007,15,11,0
Keke a Nico Rosbergovi,1986,2006,19,12,0
Gilles a Jacques Villeneuvovi,1982,1996,13,9,0
Mario a Michael Andretti,1982,1993,10,8,5


In [40]:
fsFrame.to_csv("pary.csv")

In [41]:
sonsFrame.to_csv("synove.csv")

In [42]:
import re
def cleanhtml(input):
    input = re.sub("\\n\s*","",input)
    input = re.sub(" border=\"1\"","",input)
    input = re.sub("\.0", "",input)
    input = re.sub("\.5",",5",input)
    return input

In [43]:
cleanhtml(fsFrame.to_html(classes="table table--responsive table--w100p table--striped-red table--plain"))

'<table class="dataframe table table--responsive table--w100p table--striped-red table--plain"><thead><tr style="text-align: right;"><th></th><th>derniéra otce</th><th>debut syna</th><th>sezón rozestupu</th><th>společných okruhů</th><th>společných soupeřů</th></tr></thead><tbody><tr><th>Michael a Mick Schumacherovi</th><td>2012</td><td>2021</td><td>8</td><td>18</td><td>8</td></tr><tr><th>Jonathan a Jolyon Palmerovi</th><td>1989</td><td>2016</td><td>26</td><td>10</td><td>0</td></tr><tr><th>Jos a Max Verstappenovi</th><td>2003</td><td>2015</td><td>11</td><td>15</td><td>3</td></tr><tr><th>Jan a Kevin Magnussenovi</th><td>1998</td><td>2014</td><td>15</td><td>14</td><td>0</td></tr><tr><th>Nelson a Nelsinho Piquetovi</th><td>1991</td><td>2008</td><td>16</td><td>11</td><td>0</td></tr><tr><th>Manfred a Markus Winkelhockovi</th><td>1985</td><td>2007</td><td>21</td><td>1</td><td>0</td></tr><tr><th>Satoru a Kazuki Nakajimovi</th><td>1991</td><td>2007</td><td>15</td><td>11</td><td>0</td></tr><tr><

In [44]:
cleanhtml(sonsFrame.to_html(classes="table table--responsive table--w100p table--striped-red table--plain"))

'<table class="dataframe table table--responsive table--w100p table--striped-red table--plain"><thead><tr style="text-align: right;"><th></th><th>závodů</th><th>titulů</th><th>bodů</th><th>vítězství</th><th>poles</th><th>top výsledek</th></tr></thead><tbody><tr><th>Max Verstappen</th><td>164</td><td>2</td><td>2008,5</td><td>36</td><td>21</td><td>1.</td></tr><tr><th>Nico Rosberg</th><td>206</td><td>1</td><td>1594,5</td><td>23</td><td>30</td><td>1.</td></tr><tr><th>Damon Hill</th><td>122</td><td>1</td><td>360</td><td>22</td><td>20</td><td>1.</td></tr><tr><th>Jacques Villeneuve</th><td>165</td><td>1</td><td>235</td><td>11</td><td>13</td><td>1.</td></tr><tr><th>Kevin Magnussen</th><td>143</td><td>0</td><td>179</td><td>0</td><td>0</td><td>2.</td></tr><tr><th>Hans-Joachim Stuck</th><td>81</td><td>0</td><td>29</td><td>0</td><td>0</td><td>3.</td></tr><tr><th>Mick Schumacher</th><td>44</td><td>0</td><td>12</td><td>0</td><td>0</td><td>6.</td></tr><tr><th>Christian Fittipaldi</th><td>43</td><td>0